In [ ]:
from training_script import training_loop
import h5py
from mymodels import CoarseNet
import os
import torch 
import skimage.io as skio
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
def get_print_images_callback(fixed_idx,num_random):
    def print_images_callback(model,dataset):
        print("Performance on random pics")
        print("##"*50)
        random_pic=fixed_idx
        num_samples=dataset["hazy_image"].shape[0]
        for j in range(0,num_random+1):
            t_batch=dataset["trans_map"][random_pic,:,:]
            h_batch=dataset["hazy_image"][random_pic:random_pic+1,:,:,:]
            t_pred=model(torch.from_numpy(h_batch))
            t_pred_npy=t_pred.detach().numpy()[0,:,:]
            plt.figure()
            _=skio.imshow(t_pred_npy)
            plt.show()
            print("##"*50)
            random_pic=np.random.randint(0,num_samples)
    return print_images_callback

In [ ]:
args={
    "num_epochs":50,
    "checkpoint":10,
    "grad_sim":True,
    "scaling_coefficient":1000,
    "patch_size":1,
    "batch_size":100,
    "lr_initial":0.01,
    # for our experiments we use a fixed LR
    "lr_decay_factor":0.1,
    "lr_decay_interval":100,
    "momentum":0.9,
    "l2_weight_decay":5e-04,
    "run_dir":"./saved_models/test"
    }

In [ ]:
dir_path=args["run_dir"]
if os.path.exists(dir_path):
    assert(os.path.isdir(dir_path))
else:
    os.makedirs(dir_path)

In [ ]:
logfilename=args["run_dir"]+"/log.txt"
with open(logfilename,"a+") as fi:
    fi.write(str(args)+"\n")
dataset = h5py.File("../data/nyu_hazy_trn.mat","r")
model=CoarseNet()

In [ ]:
training_loop(model=model,dataset=dataset,
              training_callback=get_print_images_callback(10,2),**args)